# Importing the libraries

In [61]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import time

# API key and ticker symbol

In [62]:
api_key = 'HDHS7RJSLYNN07J7'
ticker = 'TSLA'

# Importing the dataset

In [63]:
ts = TimeSeries(key = api_key, output_format = 'pandas')
data, meta_data = ts.get_intraday(symbol = ticker, interval = '1min', outputsize = 'full')
print(data.head())

                     1. open  2. high   3. low  4. close  5. volume
date                                                               
2020-07-14 20:00:00  1568.47  1568.47  1566.00   1566.03     2818.0
2020-07-14 19:59:00  1568.00  1569.00  1567.00   1568.00     4457.0
2020-07-14 19:58:00  1569.29  1569.29  1569.29   1569.29      982.0
2020-07-14 19:57:00  1568.00  1569.00  1568.00   1569.00     3095.0
2020-07-14 19:56:00  1570.75  1570.75  1570.75   1570.75     1086.0


# Basic patterns

## Sentiment (Bearish or Bullish)

In [64]:
color = []
for ind in data.index:
    if data['1. open'][ind] > data['4. close'][ind]:
        color.append('red') #open higher than close --> bearishness
    else:
        color.append('blue') #open is lower than or equal to close --> bulllishness
data['color'] = color
print(data.head())

                     1. open  2. high   3. low  4. close  5. volume color
date                                                                     
2020-07-14 20:00:00  1568.47  1568.47  1566.00   1566.03     2818.0   red
2020-07-14 19:59:00  1568.00  1569.00  1567.00   1568.00     4457.0  blue
2020-07-14 19:58:00  1569.29  1569.29  1569.29   1569.29      982.0  blue
2020-07-14 19:57:00  1568.00  1569.00  1568.00   1569.00     3095.0  blue
2020-07-14 19:56:00  1570.75  1570.75  1570.75   1570.75     1086.0  blue


## Range

In [65]:
range = []
for ind in data.index:
    range.append(data['2. high'][ind] - data['3. low'][ind])
data['range'] = range
print(data.head())

                     1. open  2. high   3. low  4. close  5. volume color  \
date                                                                        
2020-07-14 20:00:00  1568.47  1568.47  1566.00   1566.03     2818.0   red   
2020-07-14 19:59:00  1568.00  1569.00  1567.00   1568.00     4457.0  blue   
2020-07-14 19:58:00  1569.29  1569.29  1569.29   1569.29      982.0  blue   
2020-07-14 19:57:00  1568.00  1569.00  1568.00   1569.00     3095.0  blue   
2020-07-14 19:56:00  1570.75  1570.75  1570.75   1570.75     1086.0  blue   

                     range  
date                        
2020-07-14 20:00:00   2.47  
2020-07-14 19:59:00   2.00  
2020-07-14 19:58:00   0.00  
2020-07-14 19:57:00   1.00  
2020-07-14 19:56:00   0.00  


## Shadow length

In [69]:
u_shadow = []
l_shadow = []
for ind in data.index:
    if data['color'][ind] == 'red':
        u_shadow.append(data['2. high'][ind] - data['1. open'][ind])
        l_shadow.append(data['4. close'][ind] - data['3. low'][ind])
    else:
        u_shadow.append(data['2. high'][ind] - data['4. close'][ind])
        l_shadow.append(data['1. open'][ind] - data['3. low'][ind])
data['u_shadow'] = u_shadow
data['l_shadow'] = l_shadow
print(data.head())

                     1. open  2. high   3. low  4. close  5. volume color  \
date                                                                        
2020-07-14 20:00:00  1568.47  1568.47  1566.00   1566.03     2818.0   red   
2020-07-14 19:59:00  1568.00  1569.00  1567.00   1568.00     4457.0  blue   
2020-07-14 19:58:00  1569.29  1569.29  1569.29   1569.29      982.0  blue   
2020-07-14 19:57:00  1568.00  1569.00  1568.00   1569.00     3095.0  blue   
2020-07-14 19:56:00  1570.75  1570.75  1570.75   1570.75     1086.0  blue   

                     range  u_shadow  l_shadow  
date                                            
2020-07-14 20:00:00   2.47       0.0      0.03  
2020-07-14 19:59:00   2.00       1.0      1.00  
2020-07-14 19:58:00   0.00       0.0      0.00  
2020-07-14 19:57:00   1.00       0.0      0.00  
2020-07-14 19:56:00   0.00       0.0      0.00  


## Upper bound and lower bound

In [72]:
ub = []
lb = []
for ind in data.index:
    if data['color'][ind] == 'blue':
        ub.append(data['4. close'][ind])
        lb.append(data['1. open'][ind])
    else:
        ub.append(data['1. open'][ind])
        lb.append(data['4. close'][ind])
data['u_bond'] = ub
data['l_bound'] = lb
print(data.head())

                     1. open  2. high   3. low  4. close  5. volume color  \
date                                                                        
2020-07-14 20:00:00  1568.47  1568.47  1566.00   1566.03     2818.0   red   
2020-07-14 19:59:00  1568.00  1569.00  1567.00   1568.00     4457.0  blue   
2020-07-14 19:58:00  1569.29  1569.29  1569.29   1569.29      982.0  blue   
2020-07-14 19:57:00  1568.00  1569.00  1568.00   1569.00     3095.0  blue   
2020-07-14 19:56:00  1570.75  1570.75  1570.75   1570.75     1086.0  blue   

                     range  u_shadow  l_shadow   u_bond  l_bound  
date                                                              
2020-07-14 20:00:00   2.47       0.0      0.03  1568.47  1566.03  
2020-07-14 19:59:00   2.00       1.0      1.00  1568.00  1568.00  
2020-07-14 19:58:00   0.00       0.0      0.00  1569.29  1569.29  
2020-07-14 19:57:00   1.00       0.0      0.00  1569.00  1568.00  
2020-07-14 19:56:00   0.00       0.0      0.00  1570.75  1

# Single candlestick patterns

## Marubozu

In [91]:
type = []
data_dict = data.to_dict('records')
for r in data_dict:
    if (r('u_shadow')/r('u_bound') <= 0.003 and 
        r('l_shadow')/r('l_bound') <= 0.003):
        type.append('marubozu')

TypeError: 'dict' object is not callable